## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/97-googleq-a-train-bert-base-misspellings/model.h5'
BERT_PATH = '/kaggle/input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12'
TOKENIZER_PATH = BERT_PATH+'/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(TOKENIZER_PATH, True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

x = GlobalAveragePooling1D()(sequence_output)
x = Dropout(0.2)(x)
output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.959495,0.786367,0.559448,0.555869,0.376218,0.496313,0.615161,0.600962,0.451928,...,0.941062,0.924121,0.568336,0.981495,0.969725,0.883437,0.021166,0.021651,0.902222,0.933777
1,46,0.919867,0.528260,0.008883,0.732770,0.761513,0.912134,0.534445,0.517123,0.109252,...,0.762035,0.952253,0.625565,0.973913,0.982748,0.857086,0.952797,0.090200,0.064361,0.880821
2,70,0.923633,0.679792,0.057819,0.771727,0.905932,0.891563,0.510589,0.566562,0.065650,...,0.875242,0.896895,0.627088,0.976193,0.954455,0.793658,0.039139,0.059254,0.782990,0.907814
3,132,0.884650,0.432251,0.008940,0.591237,0.800554,0.908693,0.555661,0.543723,0.082383,...,0.751526,0.920708,0.632549,0.968671,0.978524,0.856955,0.887395,0.151203,0.275847,0.877663
4,200,0.943845,0.472734,0.066423,0.816633,0.763531,0.826733,0.589145,0.633191,0.049524,...,0.788914,0.903499,0.598381,0.972959,0.967688,0.800297,0.148866,0.126334,0.403524,0.931478


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.910321,0.628418,0.049087,0.697873,0.803859,0.835520,0.563735,0.511823,0.207683,...,0.838643,0.916192,0.664465,0.966787,0.969969,0.853817,0.529961,0.143091,0.474696,0.909930
std,2812.670060,0.043162,0.139717,0.078883,0.120473,0.110523,0.145895,0.049197,0.086024,0.170419,...,0.074472,0.033570,0.062501,0.012623,0.015214,0.050396,0.351352,0.078651,0.288910,0.025413
min,39.000000,0.707100,0.369026,0.003884,0.259045,0.352013,0.199107,0.427565,0.349855,0.003513,...,0.632107,0.718163,0.521222,0.905192,0.881369,0.660881,0.003578,0.006858,0.006060,0.819715
25%,2572.000000,0.883039,0.505529,0.010560,0.617424,0.766216,0.810882,0.532102,0.451604,0.078001,...,0.780518,0.896106,0.623404,0.959654,0.962940,0.825818,0.149050,0.085220,0.225076,0.893537
50%,5093.000000,0.915328,0.611862,0.017350,0.688910,0.821550,0.888497,0.560205,0.490657,0.150357,...,0.843119,0.921130,0.660329,0.967906,0.972581,0.859197,0.586846,0.134838,0.458586,0.911023
75%,7482.000000,0.943735,0.741345,0.047895,0.778522,0.875651,0.925682,0.593479,0.551626,0.293118,...,0.901892,0.940845,0.701893,0.975211,0.980758,0.888463,0.867071,0.197291,0.718159,0.928271
max,9640.000000,0.982777,0.950601,0.560982,0.985189,0.983168,0.988383,0.739721,0.824141,0.806368,...,0.972392,0.983507,0.845791,0.992384,0.992992,0.963724,0.992394,0.400044,0.988123,0.963710
